<a href="https://colab.research.google.com/github/vbipin/aip/blob/master/graph_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#ref: https://algs4.cs.princeton.edu/41graph/

In [0]:
class Edge :
    def __init__(self, src, dest, weight=1) :
        self.src    = src
        self.dest   = dest
        self.weight = float(weight)
    #for printing
    def __repr__(self) :
        return str((self.src, self.dest, self.weight))
    
class Graph : 
    def __init__(self) :
        self.V=0
        self.E=0
        self.nodes = {} #{ index: adjlist }
    
    def adj(self, v) :
        return self.nodes.get(v, [])
    
    def add_vertex(self, v) : #v is the new vertex index
        if v not in self.nodes :
            self.V += 1
            self.nodes[v] = [] #empty adj list
            
    def add_edge(self, src, dest, weight=1 ) : #edge is a tuple of (src, dest [weight])
        self.E += 1  
        self.nodes[src].append(Edge(src, dest, weight))
        
    def __str__(self) :
        return str(self.nodes)

In [0]:
#recursive dfs      
def dfs(g, start, end, visited=None) : 
    """returns True if found a path from start to end, else False"""
    visited = visited or {} #only for the first time

    if start == end :
        return True

    visited[start] = True #we mark it as visited

    for edge in g.adj(start) :  #we look at all the nodes next to the current
        if edge.dest not in visited : #we search if we havent visted them yet
            if dfs(g, edge.dest, end, visited) :
                return True

    return False   

In [0]:
#Non recursive dfs with paths
#We use a list as stack        
def dfs_path(g, start, end) : 
    """returns path if found a path from start to end, else []"""
    marked = {}
    stack   = []

    stack.append((start, [start])) #we are queing the index and path
    marked[start] = True #we mark it as visited
    
    while stack : #not empty
        current, path = stack.pop() #NOTE: This is what makes it a stack
        if current == end :
            return path

        for edge in g.adj(current) :  #we look at all the nodes next to the current
            if edge.dest not in marked : #we search if we havent visted them yet
                stack.append( (edge.dest, path+[edge.dest]) )
                marked[edge.dest] = True #we mark it as visited

    return []

In [0]:
#In bfs we simply use a queue instead of a stack
def bfs_path(g, start, end) : 
    """returns path if found a path from start to end, else False"""
    marked = {}
    queue   = [] #use pop(0) to make it as a queue
    
    queue.append((start, [start])) #we are queing the index and path
    marked[start] = True #we mark it as visited
    
    while queue : #not empty
        current, path = queue.pop(0)  #NOTE: This is what makes it a queue
        if current == end :
            return path

        for edge in g.adj(current) :  #we look at all the nodes next to the current
            if edge.dest not in marked : #we search if we havent visted them yet
                queue.append( (edge.dest, path+[edge.dest]) )
                marked[edge.dest] = True #we mark it as visited

    return []

In [0]:
#for ufs we need a priority queue.
#we need to pop the items according to its priority
#lets try to implement with a list of (priority, node)
#This class use both a hap and dict
#The dict is used to check membership and to retrieve the node.

from heapq import heappush, heappop
#ref: https://docs.python.org/3.0/library/heapq.html

class PQ :
    def __init__(self) :
        self.q = []
        self.nodes = {} #we just keep the nodes in q here. This is used to check if a node is already present.
        self.count = 0  #just to avoid class comparisons if priorities are equal.
        
    def __len__(self) :
        return len(self.q)
    
    def __getitem__(self, node) :
        priority, count, node = self.nodes[node]
        return priority, node
    
    def __contains__(self, node) :
        #contains(a,b)
        #Return the outcome of the test b in a. Note the reversed operands.
        return node in self.nodes
        
    def append(self, priority, node): #high priority at the end
        self.count += 1
        record = (priority, self.count, node) #this is the one we store in PQ
        #print(record) #XXX
        heappush(self.q, record)        
        self.nodes[node] = record
        return 
    
    def pop(self):        
        priority, count, node = heappop(self.q) 
        del self.nodes[node]  
        return priority, node

def ucs_path(g, start, end) :
    """returns the path found according to uniform cost search"""
    """
    Here we need the edge values. We keep the fringe as a priority queue.
    Otherwise, the code should look the same as other searches.
    """
    marked = {}
    parent = {}
    queue   = PQ()

    queue.append(0, start) #we are queing the index and path
    marked[start] = True #we mark it as visited
    parent[start] = None #we need this to calculate the path #XXX can we combine with marked??
    
    while queue : #not empty
        cost, current = queue.pop() 
        if current == end :
            return path_from_dict(parent, end)
        
        for edge in g.adj(current) :  #we look at all the nodes next to the current
            if edge.dest not in marked: #we search if we havent visted them yet
                queue.append( edge.weight + cost, edge.dest )
                marked[edge.dest] = True #we mark it as visited
                parent[edge.dest] = current
            elif edge.dest in queue : #is it there already?
                """
                We need to updat the queue if the new path cost is lower.
                We must replace the existing node in the PQ. But our PQ can handle mutiple nodes.
                so, we simply add the new instance with lesser path cost.
                """
                path_cost, node = queue[edge.dest]
                if path_cost > edge.weight + cost : #the current node and paths are better
                    queue.append( edge.weight + cost, edge.dest )
                    parent[edge.dest] = current #new path

    return []


#convenient funtion to calculate the path from dict
def path_from_dict( d, end ) :
    path = [end]
    while True :
        parent = d[end]
        if not parent:
            break
        path.insert(0, parent)
        end = parent
    return path



In [0]:
################################################################################

In [0]:
###### Now some convenient funtions to build the graphs
def directed_graph(g, edge_list) : 
    """take a stream of tuples (src, dest [weight]) and build a directed graph"""
    for src, dest, *weight in edge_list : #weight is optional
        g.add_vertex(src)
        g.add_vertex(dest)
        g.add_edge(src, dest, *weight) #add the edge one by one
    return g

def undirected_graph(g, edge_list ) : 
    """take a stream of tuples (src, dest [weight]) and build a undirected graph"""
    for src, dest, *weight in edge_list : #weight is optional
        g.add_vertex(src)
        g.add_vertex(dest)
        g.add_edge(src, dest, *weight)
        g.add_edge(dest, src, *weight) #reverse the edge, so undirected
    return g

In [0]:
################################################################################

In [0]:
#https://algs4.cs.princeton.edu/41graph/images/graph.png
#ref: https://stackoverflow.com/questions/32370281/how-to-embed-image-or-picture-in-jupyter-notebook-either-from-a-local-machine-o
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://algs4.cs.princeton.edu/41graph/images/graph.png")

In [0]:
#This is the same graph as in https://algs4.cs.princeton.edu/41graph/
tiny_graph = """
0 5
4 3
0 1
9 12
6 4
5 4
0 2
11 12
9 10
0 6
7 8
9 11
5 3
"""
def text_to_edges( lines ) :
    return [ [ int(i) for i in line.split()] for line in tiny_graph.split("\n") if line and len(line.split()) > 1 ]

g = undirected_graph( Graph(), text_to_edges( tiny_graph ) )

In [0]:
dfs_path(g,2,3)

[2, 0, 6, 4, 3]

In [0]:
bfs_path(g,2,3)

[2, 0, 5, 3]

In [0]:
################################################################################

In [0]:
#We create some sample search problems to check our algorithms
#this class will create a 2D grid of row x colums and its graph.
#Some of the cells can be disabled by putting it into walls
#for example SearchGrid(3,4,walls=[0,10]) will create a grid that has cell 0 and 10 are disabled.

class SearchGrid :
    def __init__(self, rows=5, columns=5, walls=[], diagonal=False ) : #walls -> [ list of index ]
        self.rows = rows
        self.columns = columns
        self.N = rows * columns #total cells
        self.walls = walls
        self.diagonal = diagonal #falg to check if can move diagonally
        
    def edges(self) : 
        """return edges of the graph in a list of tuples (u,v)"""
        edges = []

        #Just a convenient funtion
        def _add_edge(u, v) :
            #we add the edge if source and destinations are not walls
            #and within the grid
            if v % self.columns == 0 : #check if at the edge.
                return
            if u not in self.walls and v not in self.walls and u < self.N and v < self.N :
                edges.append( (u,v) )
        
        #first the forward links to the right
        for i in range(self.N) :
            #if (i+1) % self.columns != 0 :#checking if it is an edge cell
            _add_edge(i, i+1)            #connect to next cell
            _add_edge(i,i+self.columns)  #connect to the cell below; it is i+width
            if self.diagonal : #add diagonal edges as well
                _add_edge(i,i+self.columns+1)  #connect to the cell below + 1
                #_add_edge(i,i+self.columns-1)  #connect to the cell below + 1
                
        return edges
    
    #pretty print the grid and path if given. path -> [ list of nodes ]
    def print(self, path=[]) :
        for i in range(self.N) :
            if i in self.walls :
                print('# ', end='')
            elif i in path :
                print('^ ', end='')
            else :
                print('. ', end='')
            if (i+1) % self.columns == 0 :
                print("")

In [0]:
grid = SearchGrid(3,4,walls=[10,2])
g = undirected_graph(Graph(), grid.edges() )

In [0]:
grid.print(bfs_path(g,11,9))

. . # . 
. ^ ^ ^ 
. ^ # ^ 


In [0]:
#Make a grid with walls
grid = SearchGrid(10,10,walls=[10,2,17,199])

#Make the graph of the grid from the edges
g = undirected_graph(Graph(), grid.edges() )

In [0]:
path = bfs_path(g,0,grid.N-1)
grid.print(path)

^ ^ # . . . . . . . 
# ^ ^ ^ ^ ^ ^ # . . 
. . . . . . ^ ^ ^ ^ 
. . . . . . . . . ^ 
. . . . . . . . . ^ 
. . . . . . . . . ^ 
. . . . . . . . . ^ 
. . . . . . . . . ^ 
. . . . . . . . . ^ 
. . . . . . . . . ^ 


In [0]:
path = dfs_path(g,0,grid.N-1)
grid.print(path)

^ ^ # . . . . . . . 
# ^ . . . . . # . . 
. ^ . . . . . . . . 
. ^ . . . . . . . . 
. ^ . . . . . . . . 
. ^ . . . . . . . . 
. ^ . . . . . . . . 
. ^ . . . . . . . . 
. ^ . . . . . . . . 
. ^ ^ ^ ^ ^ ^ ^ ^ ^ 


In [2]:
#Actions #just some alias
North = 0
East  = 1
South = 2
West  = 3
    
#8puzzle
class puzzleN :
    def __init__(self, start_config=None, rows=3,columns=3) : #start_config is a list with all numbers and 0
        self.rows, self.columns = rows, columns
        if start_config :
            self.data  = tuple(start_config) #never modified
            self.blank = self.data.index(0)
            self.terminal = tuple(sorted(self.data))
        self.all_actions = [North, East, South, West]
    
    def edge_cost(self, a=None, next_node=None) :
        """let us put a heuristic here"""
        #sum up all the out of alignment slides
        return 1 #just a dummy for testing
    
    def heuristic_cost(self, a=None, next_node=None) :
        """let us put a heuristic here"""
        #sum up all the out of alignment slides
        #return 1 #just a dummy for testing
        count = 0
        for i in range(len(self.data)) :
            if self.data[i] != self.terminal[i] :
                count += 1
        return count
        
    def actions(self) :
        """find the actions possible"""
        #we assume we are moving the blank to North, South, East, West if possible
        return [ a for a in self.all_actions if self.can_move(a) ]
    
    def move(self, action) :
        """make the move and return new class. The current class is not modified"""
        assert self.can_move(action) #Illegal Move: Cannot make this move.
        row, col = self.row_col(self.blank)
        if action == North :
            row -= 1
        elif action == South :
            row += 1
        elif action == East :
            col += 1
        elif action == West :
            col -= 1
        new_blank = self.position(row,col)
        #print (row,col)
        #print(new_blank)
        #we need to put zero at this new location and put the old value tot he current blank
        new_data = list(self.data)
        new_data[ self.blank ] = new_data[ new_blank ]
        new_data[ new_blank ]  = 0
        return puzzleN(new_data, self.rows, self.columns)
    
    def is_terminal(self) :
        if self.data == self.terminal :
            return True
        return False
    
    def children(self) : #reurns new states of all legal actions
        act = self.actions()
        if not act :
            return None, None
        child_nodes = [ self.move(a) for a in act ]
        return [ (c, a, self.edge_cost()) for c,a in zip(child_nodes,act) ]
    
    def can_move(self, action) :
        """return true if the 0 can take the action"""
        row, col = self.row_col(self.blank)
        #print(row,col)
        if action is North and row == 0 :           #first row, cannot move north
            return False
        if action is South and row == self.rows-1 : #last row, cannot move south
            return False        
        if action is East and col == self.columns-1:#last column, cannot move East
            return False
        if action is West and col == 0 :            #first column, cannot move West
            return False
        return True
        
    #some utility funtions. We dont need them if using numpy
    def row_col(self, i) :
        row = i // self.columns # integer divisor
        col = i % self.columns #int reminder
        return row, col
    
    def position(self, row, col) :
        return row * self.columns + col
    
    def print(self) :
        for i in range(self.rows * self.columns) :
            print(self.data[i], end='')
            if (i+1) % self.columns == 0 :
                print("")
        print('--------')
        
    def adj(self) :
        act = self.actions()
        if not act :
            return []
        return [ self.move(a) for a in act ]
    
    def __hash__(self) :
        return hash(self.data)
    
    def __eq__(self, other) :
        return self.data == other.data            
        
###################################################################################


#some test
def str_to_list(s) :
    #takes a string of ints and returns the list
    return [ int(i) for i in s.split() ]
s = """
    1 3 7
    2 5 6
    8 0 4
    """
p = puzzleN(str_to_list(s))
p.print()
q = p.move(North)
q.print()

137
256
804
--------
137
206
854
--------


In [0]:
def astar_game_path(start, print=False) :
    """returns the path found according to uniform cost search"""
    """
    Here we need the edge values. We keep the fringe as a priority queue.
    Otherwise, the code should look the same as other searches.
    """
    marked = { start: True } #mark as visited
    parent = { start: None } #keep the parent of each node to find the path in the end
    pqueue = PQ()
    pqueue.append(0, start) #we are queing the pathcost and node

    
    while pqueue : #not empty
        
        cost, current = pqueue.pop()
        if print :
            current.print()
        
        if current.is_terminal() :
            return path_from_dict(parent, current)
                
        for dest, action, weight in current.children() :  #we look at all the nodes next to the current
            dest_cost = weight + cost + dest.heuristic_cost()
            
            if dest not in marked: #we search if we havent visted them yet
                pqueue.append( dest_cost, dest )
                marked[dest] = True #we mark it as visited
                parent[dest] = current
                
            elif dest in pqueue : #is it there already?
                """
                We need to updat the queue if the new path cost is lower.
                XXX We must replace the existing node in the PQ
                """
                path_cost, node = pqueue[dest]
                if path_cost > dest_cost : #the current node and paths are better
                    pqueue.append( dest_cost, dest ) #instead of replace, we simply add
                    parent[dest] = current #new path

    return []



In [0]:
#A hacky way to create a random initial 8puzzle
import random
def shuffle_puzzle8(N=10) :
    a = puzzleN([0,1,2,3,4,5,6,7,8])
    a.move(East).move(South)
    for _ in range(N) :
        act = a.actions()
        #print(act)
        if act:
            a = a.move(act[random.randint(0,len(act)-1)])
    return a

In [0]:
############################################################################

In [6]:
p = shuffle_puzzle8(40)
p.print()

740
362
185
--------


In [9]:
path = astar_game_path(p)
print(len(path))

19


In [0]:
###########################################################################

In [0]:
#This class will provide the game of RushHour
#ref: https://en.wikipedia.org/wiki/Rush_Hour_(puzzle)
#the parking lot is represented as a 2D grid with empty space has value 0
#any number above 0 is a car. A car that occupies two cells have its number in both the cells

#Actions are specified as a tuple of (car Number, North/East/South/West) eg(1, North)
#The red car to bring out is 1,1
#e.g.
"""
0 0 0 0 0 0
5 0 0 2 0 0
5 1 1 2 0 0
0 0 4 4 4 0
0 0 3 3 0 0
0 0 0 0 0 0
"""

#just some alias
North = 0
East  = 1
South = 2
West  = 3
class RushHour :
    def __init__(self, start_config, rows=6, columns=6) : #out_row is the row where the red car is
        self.rows, self.columns = rows, columns
        self.N    = rows * columns
        self.data = tuple(start_config)
        #we fix this now
        self.red_car = 1
        self.out_row = 2 #we can find it out from the data; XXXX
        self.cars  = self._cars() #dictionary of { car_num : (direction, start_index, end_index)...}
        
    def is_terminal(self) :
        """checks if the current config is the end or not"""
        #it is end when the redcar reaches the end of the out_row
        exit_index = (self.out_row+1)*self.columns - 1 #we go one ahead and -1 to reach the end of out_row
        if self.data[exit_index] == self.red_car :
            return True
        return False
    
    def actions(self) :
        return [ (c,a) for c in self.cars.keys() for a in [North, East, South, West] if self.can_move((c,a))]
    
    def move(self, action) : #will return new class. Existing one is not changed
        car, act = action
        direction, start_idx, end_idx = self.cars[car] #we take out the car indexs
        #new data; we dont touch the original
        new_data = list(self.data)
        
        if act == North :
            new_start_idx = start_idx - self.columns            
            new_data[new_start_idx], new_data[end_idx]  = car, 0
            
        elif act == South :
            new_end_idx = end_idx + self.columns
            new_data[start_idx], new_data[new_end_idx]  = 0, car
            
        elif act == East :           
            new_end_idx = end_idx + 1
            new_data[start_idx], new_data[new_end_idx]  = 0, car
        
        elif act == West :            
            new_start_idx = start_idx - 1
            new_data[new_start_idx], new_data[end_idx]  = car, 0
            
        return RushHour(new_data, self.rows, self.columns)
    
    
    def children(self) :
        """returns list of tuple( nextnode, action, weight )"""
        if self.is_terminal() :
            return []
        #we need to find which all cars can move. We sweep from 0 to end.
        return [ (self.move(a), a, 1) for a in self.actions() ]
    
    def heuristic_cost(self) :
        return 1 #dummy
    
    def __hash__(self) :
        return hash(self.data)
    
    def __eq__(self, other) :
        return self.data == other.data
    
    def print(self) :
        for i in range(self.N) :
            print(self.data[i], end=' ')
            if (i+1) % self.columns == 0 :
                print("")
        print('------------')
    
    #-------
    #New funtion that finds all cars in one pass; used only in init.
    #cars are stored as (direction, start index, end index)
    #direction==0 means horizontal car
    def _cars(self) :
        cars = {}
        for i, car in enumerate(self.data) : #we go through the list
            if car in cars :
                d, s, e = cars[car] #previous start and end index
                #d == 0 initially; we need to make it 1 if vertical car
                if i-s >= self.columns :
                    d = 1
                cars[car] = (d, s, i) #replace with new index
            elif car > 0 : #new car, 0 and -1 we dont consider as cars
                cars[car] = (0,i,i) #initially we put direction as horizontal.
        return cars
    
    def can_move(self, action) :
        car, act = action
        direction, start_idx, end_idx = self.cars[car]
        if direction == 0 : #horizontal car
            if act == East : #can we move to right?
                new_end_idx = end_idx + 1
                #if not at the right edge and there is space
                if new_end_idx < self.N and new_end_idx % self.columns != 0 and self.data[new_end_idx] == 0: 
                    return True
            if act == West : #can we move to left?
                new_start_idx = start_idx - 1
                if new_start_idx >= 0 and start_idx % self.columns != 0 and self.data[new_start_idx] == 0: 
                    return True
        else : #it is a vertical car
            if act == North : #can we move up?
                new_start_idx = start_idx - self.columns
                #if not at the top edge and there is space
                if new_start_idx >= 0 and self.data[new_start_idx] == 0: 
                    return True
            if act == South : #can we move down?
                new_end_idx = end_idx + self.columns
                if new_end_idx < self.N and self.data[new_end_idx] == 0: 
                    return True
            
        return False

In [0]:
s = """
0 0 0 0 7 7
5 0 0 2 6 0
5 1 1 2 6 0
0 0 4 4 4 0
0 0 3 3 0 0
0 0 0 0 0 0
"""
r = RushHour(str_to_list(s))

In [12]:
path = astar_game_path(r)
print(len(path))

8


In [0]:

move15 = """
2 2 0 3 0 0
0 4 0 3 5 5
6 4 1 1 7 8
6 9 9 0 7 8
0 0 10 11 11 0
12 12 10 0 0 0
"""

In [14]:
h = RushHour(str_to_list(move15))
path = astar_game_path(h, print=False)
print(len(path))

28


In [0]:
hard = """
2 5 5 0 8 0
2 6 7 0 8 9
2 6 7 1 1 9
3 3 3 10 0 9
0 0 11 10 13 13
4 4 11 12 12 0
"""

In [16]:
h = RushHour(str_to_list(hard))
path = astar_game_path(h, print=False)
print(len(path))

82
